# Regression Notebook

## Imports

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import seaborn as sns
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import RFE
from numpy.polynomial.polynomial import polyfit
from pandas import Series
import matplotlib.pylab as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [1]:
!pwd

/Users/flatironschool/Documents/NBA Salaries Reg


## Load CSV Data

In [326]:
play_facts = pd.read_csv('/Users/flatironschool/Documents/NBA Salaries Reg/Players.csv')
play_sal = pd.read_csv('/Users/flatironschool/Documents/NBA Salaries Reg/NBA_season1718_salary.csv')
play_stats = pd.read_csv('/Users/flatironschool/Documents/NBA Salaries Reg/Seasons_Stats.csv')

## Custom columns and data cleaning

In [462]:
#play_facts.drop(['birth_city','birth_state'],axis=1,inplace=True)
#play_facts.head(2)

### MAYBE FOR LATER

In [329]:
#play_sal.rename(columns={'season17_18':'salary'},inplace=True)
salary_dict = {}
for i in range(len(play_sal)):
    salary_dict[play_sal.Player[i]]=play_sal.salary[i]

In [330]:
#play_stats_17=play_stats[play_stats.Year==2017]
#play_stats_17['Start%']=play_stats_17['GS']/play_stats_17['G']
#play_stats_17['ppg']=play_stats_17['PTS']/play_stats_17['G']
#play_stats_17['spg']=play_stats_17['STL']/play_stats_17['G']
#play_stats_17['apg']=play_stats_17['AST']/play_stats_17['G']
#play_stats_17['rpg']=play_stats_17['TRB']/play_stats_17['G']
#play_stats_17['bpg']=play_stats_17['BLK']/play_stats_17['G']
#play_stats_17['fpg']=play_stats_17['PF']/play_stats_17['G']
#play_stats_17['tpg']=play_stats_17['TOV']/play_stats_17['G']
#play_stats_17['mpg']=play_stats_17['MP']/play_stats_17['G']

In [331]:
#cancel_list= ['PTS','PF','BLK','TOV','BLK','STL','AST','TRB','DRB','ORB','FTA','FT','2PA','2P','3PA','3P','FGA','FG','OBPM','DBPM','blank2','WS','DWS','OWS','TOV%','BLK%','STL%','AST%','DRB%','ORB%','FTr','3PAr','Tm','GS']

In [332]:
#play_stats_17.drop(cancel_list,axis=1,inplace=True)
#play_stats_17['salary']=0

In [333]:
#play_stats_172 = play_stats_17.set_index('Player')

In [335]:
#play_stats_173 = play_stats_172[play_stats_172.index.value_counts()<2]

In [336]:
print(play_stats_173.shape)
print(len(salary_dict))

(433, 29)
535


In [337]:
play_stats_173.reset_index(inplace=True)
play_stats_173.head()

,Player,Unnamed: 0,Year,Pos,Age,G,MP,PER,TS%,TRB%,...,Start%,ppg,spg,apg,rpg,bpg,fpg,tpg,mpg,salary
0,Alex Abrines,24096,2017.0,SG,23.0,68.0,1055.0,10.1,0.560,4.5,...,0.088235,5.970588,0.544118,0.588235,1.264706,0.117647,1.676471,0.485294,15.514706,0
1,Steven Adams,24100,2017.0,C,23.0,80.0,2389.0,16.5,0.589,14.2,...,1.000000,11.312500,1.100000,1.075000,7.687500,0.975000,2.437500,1.825000,29.862500,0
2,Arron Afflalo,24101,2017.0,SG,31.0,61.0,1580.0,9.0,0.559,4.6,...,0.737705,8.442623,0.344262,1.278689,2.049180,0.114754,1.704918,0.688525,25.901639,0
3,Alexis Ajinca,24102,2017.0,C,28.0,39.0,584.0,12.9,0.529,16.0,...,0.384615,5.307692,0.512821,0.307692,4.538462,0.564103,1.974359,0.794872,14.974359,0
4,Cole Aldrich,24103,2017.0,C,28.0,62.0,531.0,12.7,0.549,17.4,...,0.000000,1.693548,0.403226,0.403226,2.548387,0.370968,1.370968,0.274194,8.564516,0


In [338]:
def add_salary(player):
    try:
        s = salary_dict[player]
        return s
    except:
        s = 0
        return s

In [340]:
#play_stats_173.salary = play_stats_173.Player.map(add_salary)

In [341]:
### try - except should be tied to a specific error

## Remove target from feature DataFrame

In [379]:
feature_list = list(play_stats_173.columns[0:29])

In [380]:
df_features = play_stats_173.iloc[:,0:29]
df_target = pd.DataFrame(play_stats_173['salary'])

In [381]:
### USING NBA INFO TO FILL NA

# 3 - 0.362
# FT - 0.767
# 2 - 0.53
# eFG - 0.521
# FG - 0.46

df_features['3P%'].fillna(0.362,inplace=True)
df_features['FT%'].fillna(0.767,inplace=True)
df_features['2P%'].fillna(0.581,inplace=True)
df_features['eFG%'].fillna(0.521,inplace=True)
df_features['FG%'].fillna(0.46,inplace=True)
df_features['TS%'].fillna(0.54,inplace=True)

In [382]:
#df_features.drop(['blanl','Year','Unnamed: 0'],axis=1,inplace=True)
#df_features.set_index('Player',inplace=True)

In [383]:
df_features.drop('Pos',axis=1,inplace=True)

In [384]:
#df_features.dropna(axis=1,inplace=True)
#df_target.dropna(axis=1,inplace=True)

In [385]:
print(df_features.shape[0]==df_target.shape[0])

True


## Check messy model

In [386]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(df_features, df_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [387]:
yhat = linreg.predict(df_features)
SS_Residual = np.sum((df_target-yhat)**2)
SS_Total = np.sum((df_target-np.mean(df_target))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(df_target)-1)/(len(df_target)-df_features.shape[1]-1)

In [388]:
r_squared

salary    0.64966
dtype: float64

In [390]:
#### WHY DOES THIS NOT WORK?
#mod = sm.OLS(df_target,df_features).fit()
#mod.summary()

## Feature engineering (remove correlation too)

In [391]:
df_pos = pd.get_dummies(play_stats_173.Pos)

In [392]:
df_pos.drop('C',axis=1,inplace=True)

In [393]:
df_pos.head(2)

,PF,PG,SF,SG
0,0,0,0,1
1,0,0,0,0


In [396]:
# no categorical features here
df_features.head(2)

,Age,G,MP,PER,TS%,TRB%,USG%,WS/48,BPM,VORP,...,FT%,Start%,ppg,spg,apg,rpg,bpg,fpg,tpg,mpg
Player,,,,,,,,,,,,,,,,,,,,,
Alex Abrines,23.0,68.0,1055.0,10.1,0.560,4.5,15.9,0.095,-2.5,-0.1,...,0.898,0.088235,5.970588,0.544118,0.588235,1.264706,0.117647,1.676471,0.485294,15.514706
Steven Adams,23.0,80.0,2389.0,16.5,0.589,14.2,16.2,0.130,0.5,1.5,...,0.611,1.000000,11.312500,1.100000,1.075000,7.687500,0.975000,2.437500,1.825000,29.862500


In [425]:
#normality check and standard scale
df_features.columns

Index(['Age', 'G', 'MP', 'PER', 'TS%', 'TRB%', 'USG%', 'WS/48', 'BPM', 'VORP',
       'FG%', '3P%', '2P%', 'eFG%', 'FT%', 'Start%', 'ppg', 'spg', 'apg',
       'rpg', 'bpg', 'fpg', 'tpg', 'mpg'],
      dtype='object')

In [426]:
print('not normal: age, g, mp, trb, vorp, 3p%, start%, ppg, apg, spg, rpg, bpg, tpg')

not normal: age, g, mp, trb, vorp, 3p%, start%, ppg, apg, spg, rpg, bpg, tpg


In [427]:
# process: 1. standard scale  2. box-cox  3. combine categorical

In [430]:
# step 1
ss = StandardScaler()
df_features_scaled = pd.DataFrame(ss.fit_transform(df_features))
df_features_scaled.columns = df_features.columns
df_features_scaled.head()

,Age,G,MP,PER,TS%,TRB%,USG%,WS/48,BPM,VORP,...,FT%,Start%,ppg,spg,apg,rpg,bpg,fpg,tpg,mpg
0,-0.776774,0.557056,-0.224825,-0.504898,0.366361,-1.087612,-0.488534,0.195658,-0.193764,-0.511420,...,1.149238,-0.749744,-0.417225,-0.218279,-0.706647,-0.952704,-0.675624,-0.008540,-0.781187,-0.490403
1,-0.776774,1.040757,1.339122,0.576797,0.677850,0.755024,-0.435795,0.600906,0.491879,0.556749,...,-1.000842,1.498374,0.442531,1.111278,-0.438459,1.666141,1.462862,1.011098,0.894800,1.049997
2,1.046224,0.274898,0.390671,-0.690814,0.355620,-1.068616,-0.752227,-0.406425,-0.422312,-0.845223,...,1.104288,0.851639,-0.019363,-0.696292,-0.326235,-0.632841,-0.682840,0.029574,-0.526944,0.624754
3,0.362600,-0.611887,-0.777014,-0.031656,0.033389,1.096956,-0.259999,0.021980,-0.559440,-0.644941,...,-0.146804,-0.018966,-0.523915,-0.293135,-0.861215,0.382145,0.437965,0.390575,-0.393902,-0.548416
4,0.362600,0.315206,-0.839149,-0.065459,0.248209,1.362904,-1.631206,0.438807,0.514733,-0.177617,...,-0.468941,-0.967304,-1.105595,-0.555263,-0.808580,-0.429293,-0.043769,-0.417858,-1.045276,-1.236586


In [434]:
# step 2
df_features_scaled['Age']=list(stats.boxcox(abs(df_features_scaled['Age'])))[0]
df_features_scaled['G']=list(stats.boxcox(abs(df_features_scaled['G'])))[0]
df_features_scaled['MP']=list(stats.boxcox(abs(df_features_scaled['MP'])))[0]
df_features_scaled['TRB%']=list(stats.boxcox(abs(df_features_scaled['TRB%'])))[0]
df_features_scaled['VORP']=list(stats.boxcox(abs(df_features_scaled['VORP'])))[0]
df_features_scaled['3P%']=list(stats.boxcox(abs(df_features_scaled['3P%'])))[0]
df_features_scaled['Start%']=list(stats.boxcox(abs(df_features_scaled['Start%'])))[0]
df_features_scaled['ppg']=list(stats.boxcox(abs(df_features_scaled['ppg'])))[0]
df_features_scaled['spg']=list(stats.boxcox(abs(df_features_scaled['spg'])))[0]
df_features_scaled['apg']=list(stats.boxcox(abs(df_features_scaled['apg'])))[0]
df_features_scaled['rpg']=list(stats.boxcox(abs(df_features_scaled['rpg'])))[0]
df_features_scaled['bpg']=list(stats.boxcox(abs(df_features_scaled['bpg'])))[0]
df_features_scaled['tpg']=list(stats.boxcox(abs(df_features_scaled['tpg'])))[0]

In [439]:
df_atg = df_features_scaled.copy()
df_atg['PG'] = df_pos['PG']
df_atg['SG'] = df_pos['SG']
df_atg['SF'] = df_pos['SF']
df_atg['PF'] = df_pos['PF']

In [443]:
# This brings it all together
df_atg.head(2)

,Age,G,MP,PER,TS%,TRB%,USG%,WS/48,BPM,VORP,...,apg,rpg,bpg,fpg,tpg,mpg,PG,SG,SF,PF
0,0.039353,-0.440707,-1.358368,-0.504898,0.366361,-1.733088,-0.488534,0.195658,-0.193764,-0.412009,...,-0.331281,-0.048101,-0.370539,-0.008540,-0.236384,-0.490403,0,1,0,0
1,0.039353,0.684527,-0.120946,0.576797,0.677850,-1.090477,-0.435795,0.600906,0.491879,-0.518033,...,-0.738316,0.551675,0.402285,1.011098,-0.108982,1.049997,0,0,0,0


## What happens with less outliers?

In [636]:
df_atg_2 = df_atg.copy()
df_atg_2['salary'] = df_target['salary']
df_atg_2.shape

(433, 29)

In [637]:
df_atg_2 = df_atg_2[(np.abs(stats.zscore(df_atg_2)) < 2.5).all(axis=1)]
df_atg_2.shape

(344, 29)

In [638]:
no_outliers_features = df_atg_2.iloc[:,0:28]

In [639]:
no_outliers_target = df_atg_2.iloc[:,28:29]

In [640]:
lr = LinearRegression()
lr.fit(no_outliers_features, no_outliers_target)
y_hat_no_outliers = lr.predict(no_outliers_features)
print(f"r^2: {r2_score(no_outliers_target, y_hat_no_outliers)}")
print(f"rmse: {np.sqrt(mean_squared_error(no_outliers_target, y_hat_no_outliers))}")

r^2: 0.5759703083137657
rmse: 4187405.914610501


In [641]:
mod = sm.OLS(no_outliers_target,no_outliers_features).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 salary   R-squared (uncentered):                   0.695
Model:                            OLS   Adj. R-squared (uncentered):              0.668
Method:                 Least Squares   F-statistic:                              25.77
Date:                Thu, 02 Jan 2020   Prob (F-statistic):                    2.59e-65
Time:                        09:47:41   Log-Likelihood:                         -5779.5
No. Observations:                 344   AIC:                                  1.161e+04
Df Residuals:                     316   BIC:                                  1.172e+04
Df Model:                          28                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Age          -2.2e+06   4.12e+05     -5.339      0.000   -3.01e+06   -1.39e+06
G           5.107e+05   4.07e+05      1.256      0.210   -2.89e+05    1.31e+06
MP         -4.626e+05   4.63e+05     -0.999      0.318   -1.37e+06    4.48e+05
PER          2.96e+05   1.89e+06      0.156      0.876   -3.43e+06    4.02e+06
TS%        -2.945e+06   2.03e+06     -1.450      0.148   -6.94e+06    1.05e+06
TRB%        -1.13e+06    3.8e+05     -2.973      0.003   -1.88e+06   -3.82e+05
USG%        3.622e+05    9.2e+05      0.393      0.694   -1.45e+06    2.17e+06
WS/48       2.913e+06   1.63e+06      1.790      0.074   -2.88e+05    6.11e+06
BPM        -3.413e+05   1.15e+06     -0.296      0.768   -2.61e+06    1.93e+06
VORP       -1.114e+06   4.39e+05     -2.535      0.012   -1.98e+06   -2.49e+05
FG%         2.476e+06    1.3e+06      1.900      0.058    -8.8e+04    5.04e+06
3P%        -1.694e+05   4.15e+05     -0.408      0.683   -9.85e+05    6.47e+05
2P%        -1.454e+06   7.95e+05     -1.828      0.068   -3.02e+06    1.11e+05
eFG%        1.351e+06   1.88e+06      0.719      0.472   -2.34e+06    5.05e+06
FT%         2.719e+05   5.28e+05      0.515      0.607   -7.67e+05    1.31e+06
Start%      2.094e+06   7.62e+05      2.749      0.006    5.95e+05    3.59e+06
ppg         4.702e+05   5.48e+05      0.857      0.392   -6.09e+05    1.55e+06
spg        -1.874e+05    4.4e+05     -0.426      0.670   -1.05e+06    6.78e+05
apg        -9.392e+05   5.19e+05     -1.810      0.071   -1.96e+06     8.2e+04
rpg        -1.051e+05   4.37e+05     -0.241      0.810   -9.64e+05    7.54e+05
bpg        -3.383e+05   4.25e+05     -0.796      0.427   -1.17e+06    4.98e+05
fpg         5.997e+05   5.17e+05      1.159      0.247   -4.18e+05    1.62e+06
tpg         2046.1449   5.58e+05      0.004      0.997    -1.1e+06     1.1e+06
mpg         3.257e+06   5.87e+05      5.551      0.000     2.1e+06    4.41e+06
PG          2.348e+06    9.8e+05      2.395      0.017    4.19e+05    4.28e+06
SG           2.43e+06   1.01e+06      2.400      0.017    4.38e+05    4.42e+06
SF           4.46e+06   9.39e+05      4.751      0.000    2.61e+06    6.31e+06
PF          2.852e+06   8.41e+05      3.393      0.001     1.2e+06    4.51e+06
==============================================================================
Omnibus:                       11.496   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.349
Skew:                           0.375   Prob(JB):                      0.00208
Kurtosis:                       3.546   Cond. No.                         21.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance mat

## Train-Test-Split and fit model again on initial information

In [642]:
X_tr, X_te, y_tr, y_te = train_test_split(df_atg,df_target,test_size=0.3,random_state=45)

In [643]:
lr = LinearRegression()
lr.fit(X_tr, y_tr)
y_hat_tr = lr.predict(X_tr)
coef_df = pd.DataFrame(lr.coef_)
coef_df.columns = df_atg.columns
coef_df

,Age,G,MP,PER,TS%,TRB%,USG%,WS/48,BPM,VORP,...,apg,rpg,bpg,fpg,tpg,mpg,PG,SG,SF,PF
0,-826001.855498,480187.370894,-164487.40803,-3.265309e+06,-1.087048e+06,896704.336368,1.775972e+06,4.388426e+06,-25496.597639,-770246.825621,...,803234.0819,995894.637785,10046.529986,-900599.517114,182793.343811,5.327223e+06,-2.031037e+06,-3.411742e+06,-1.624130e+06,-1.390165e+06


In [644]:
print(f"r^2: {r2_score(y_tr, y_hat_tr)}")
print(f"rmse: {np.sqrt(mean_squared_error(y_tr, y_hat_tr))}")

r^2: 0.6527368420567959
rmse: 4572307.427521682


In [645]:
lr = LinearRegression()
lr.fit(X_tr, y_tr)
y_hat_te = lr.predict(X_te)
print(f"r^2: {r2_score(y_te, y_hat_te)}")
print(f"rmse: {np.sqrt(mean_squared_error(y_te, y_hat_te))}")

r^2: 0.6188542316723659
rmse: 4797815.332482418


In [646]:
mod = sm.OLS(y_tr,X_tr).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 salary   R-squared (uncentered):                   0.726
Model:                            OLS   Adj. R-squared (uncentered):              0.698
Method:                 Least Squares   F-statistic:                              25.99
Date:                Thu, 02 Jan 2020   Prob (F-statistic):                    1.71e-61
Time:                        10:05:03   Log-Likelihood:                         -5125.0
No. Observations:                 303   AIC:                                  1.031e+04
Df Residuals:                     275   BIC:                                  1.041e+04
Df Model:                          28                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Age         -2.22e+06   4.97e+05     -4.465      0.000    -3.2e+06   -1.24e+06
G           1.673e+05   4.87e+05      0.344      0.731    -7.9e+05    1.13e+06
MP         -9.343e+05   5.68e+05     -1.645      0.101   -2.05e+06    1.84e+05
PER         8.392e+05   1.78e+06      0.470      0.638   -2.67e+06    4.35e+06
TS%        -2.147e+06   1.66e+06     -1.294      0.197   -5.41e+06    1.12e+06
TRB%       -4.791e+05   4.52e+05     -1.060      0.290   -1.37e+06    4.11e+05
USG%        1.236e+05   8.88e+05      0.139      0.889   -1.63e+06    1.87e+06
WS/48       2.838e+06   1.53e+06      1.849      0.066   -1.84e+05    5.86e+06
BPM        -5.015e+05   7.62e+05     -0.658      0.511      -2e+06    9.99e+05
VORP       -1.167e+06   5.17e+05     -2.257      0.025   -2.19e+06   -1.49e+05
FG%         2.527e+06   1.32e+06      1.910      0.057   -7.81e+04    5.13e+06
3P%        -2.511e+05   4.14e+05     -0.606      0.545   -1.07e+06    5.64e+05
2P%        -6.579e+05   7.24e+05     -0.909      0.364   -2.08e+06    7.67e+05
eFG%       -9.254e+05   1.79e+06     -0.516      0.606   -4.45e+06     2.6e+06
FT%        -2.102e+05   4.93e+05     -0.426      0.670   -1.18e+06    7.61e+05
Start%      2.977e+06   9.84e+05      3.024      0.003    1.04e+06    4.91e+06
ppg         7.275e+05   6.11e+05      1.190      0.235   -4.76e+05    1.93e+06
spg        -3.576e+05   5.23e+05     -0.683      0.495   -1.39e+06    6.73e+05
apg        -2.494e+05    5.7e+05     -0.438      0.662   -1.37e+06    8.72e+05
rpg         3.529e+05   4.92e+05      0.717      0.474   -6.15e+05    1.32e+06
bpg        -2.589e+05   4.45e+05     -0.582      0.561   -1.13e+06    6.16e+05
fpg          3.39e+05   5.97e+05      0.568      0.570   -8.35e+05    1.51e+06
tpg         2.239e+05    6.4e+05      0.350      0.727   -1.04e+06    1.48e+06
mpg         3.945e+06   6.52e+05      6.051      0.000    2.66e+06    5.23e+06
PG           4.86e+06   1.07e+06      4.535      0.000    2.75e+06    6.97e+06
SG          4.305e+06   1.16e+06      3.715      0.000    2.02e+06    6.59e+06
SF          6.358e+06   1.17e+06      5.455      0.000    4.06e+06    8.65e+06
PF          4.596e+06   9.85e+05      4.664      0.000    2.66e+06    6.54e+06
==============================================================================
Omnibus:                        3.660   Durbin-Watson:                   1.866
Prob(Omnibus):                  0.160   Jarque-Bera (JB):                3.566
Skew:                           0.172   Prob(JB):                        0.168
Kurtosis:                       3.405   Cond. No.                         17.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance mat

# Next steps - add player info like height, weight, college